###### Imports and Settings

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [5]:
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y

# Age Well  

The following code outlines data collection, formatting, cleaning, and calculations for a data request for Age Well Tennessee.  

The following data is collected:  
+ total population 65+  
+ income  
+ caregiver for grandkids  
+ own vs rent  
+ cost burden  
+ living alone  
+ educational attainment  
+ poverty  

Most of this was directly requested, but some was added on by research staff as the request asked for other compelling data to make a case for the need for affordable housing for seniors.

In [8]:
conn = sq.connect('../../Pipeline-Census-Bureau/Outputs/CensusBureau.db')
#PL2020
sql_query = pd.read_sql('SELECT * FROM [PL_2020]', conn)
PL2020 = pd.DataFrame(sql_query)
PL2020.drop(columns = 'Source', inplace = True)
#ACS52023
sql_query = pd.read_sql('SELECT * FROM [ACS5_2023_1]', conn)
ACS52023_1 = pd.DataFrame(sql_query)
sql_query = pd.read_sql('SELECT * FROM [ACS5_2023_2]', conn)
ACS52023_2 = pd.DataFrame(sql_query)
ACS52023 = ACS52023_1.merge(ACS52023_2, on = ['NAME', 'GEO_ID', 'Source'])
ACS52023.drop(columns = 'Source', inplace = True)
dfs2023 = [ACS52023, PL2020]
twothousandtwentythree = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2023)

In [9]:
data = twothousandtwentythree.loc[(twothousandtwentythree['NAME'] == 'Tennessee')]

In [10]:
cols = ['NAME', 'Age:65 and Older', 'Age%:65 and Older',

        #med hhincome age group
        'HHIncome:Median for Householder Under 65 Over', 
        #med hhincome
        'Median Household Income',
        #household income by age group
        'HHIncome:65 Over Less than 10,000', 'HHIncome%:65 Over Less than 10,000', 
        'HHIncome:65 Over 10 to 14,999', 'HHIncome%:65 Over 10 to 14,999', 'HHIncome:65 Over 15 to 19,999', 'HHIncome%:65 Over 15 to 19,999', 
        'HHIncome:65 Over 20 to 24,999', 'HHIncome%:65 Over 20 to 24,999', 'HHIncome:65 Over 25 to 29,999', 'HHIncome%:65 Over 25 to 29,999', 
        'HHIncome:65 Over 30 to 34,999', 'HHIncome%:65 Over 30 to 34,999', 'HHIncome:65 Over 35 to 39,999', 'HHIncome%:65 Over 35 to 39,999', 
        'HHIncome:65 Over 40 to 44,999', 'HHIncome%:65 Over 40 to 44,999', 'HHIncome:65 Over 45 to 49,999', 'HHIncome%:65 Over 45 to 49,999', 
        'HHIncome:65 Over 50 to 59,999', 'HHIncome%:65 Over 50 to 59,999', 'HHIncome:65 Over 60 to 74,999', 'HHIncome%:65 Over 60 to 74,999', 
        'HHIncome:65 Over 75 to 99,999', 'HHIncome%:65 Over 75 to 99,999', 'HHIncome:65 Over 100 to 124,999', 'HHIncome%:65 Over 100 to 124,999', 
        'HHIncome:65 Over 125 to 149,999', 'HHIncome%:65 Over 125 to 149,999', 'HHIncome:65 Over 150 to 199,999', 'HHIncome%:65 Over 150 to 199,999', 
        'HHIncome:65 Over 200K or More', 'HHIncome%:65 Over 200K or More',
        #cost burden for age group
        'CBAge:Cost Burdened Households Householder 65 Over', 'CBAge%:Households Cost Burdened Householder 65 Over',
        #tenure by age group
        'Homeownership:Householders Age 60 and Over', 'Homeownership:Homeowners Age 60 and Over', 'Homeownership%:Homeowners Age 60 and Over', 
        'Homeownership%:Age 60 and Over Householders Homeowners', 'Homeownership:Renters Age 60 and Over', 'Homeownership%:Renters Age 60 and Over', 
        'Homeownership%:Age 60 and Over Householders Renters', 'Homeownership:Householders Age 65 and Over', 'Homeownership:Homeowners Age 65 and Over', 
        'Homeownership%:Homeowners Age 65 and Over', 'Homeownership%:Age 65 and Over Householders Homeowners', 'Homeownership:Renters Age 65 and Over', 
        'Homeownership%:Renters Age 65 and Over', 'Homeownership%:Age 65 and Over Householders Renters', 
        #poverty by age group
        'Poverty:Below Poverty Level 65 and Over', 'Poverty%:Below Poverty Level 65 and Over', 'Poverty%:65 and Over Below Poverty Level',
        #poverty total
        'Poverty:Population for Whom Poverty Status is Determined', 'Poverty: Population Below Poverty Level', 'Poverty%: Below Poverty Level',
        #living alone age group
        'LivingSituation:Living Alone 65 Over', 'LivingSituation%:65 Over that are Living Alone', 'LivingSituation:Living with Spouse 65 Over', 
        'LivingSituation%:65 Over that are Living with Spouse', 
        #grandparents as caregivers
        'Grandparents:Grandparents 60 Over, Primary Caregiver for Grandchild', 'Grandparents%:Grandparents 60 Over that are Primary Caregiver for Grandchild', 
        #lf stats
        'Employment:In Labor Force 65 Over', 'Employment%:Total Population 65 Over In Labor Force', 'Employment%:Population in LF 65 Over that are Unemployed', 
        #social security
        'SocialSecurity:Households with Social Security Income', 'SocialSecurity%:Households with Social Security Income'




        
            ]
data = data[cols]
data

,NAME,Age:65 and Older,Age%:65 and Older,HHIncome:Median for Householder Under 65 Over,Median Household Income,"HHIncome:65 Over Less than 10,000","HHIncome%:65 Over Less than 10,000","HHIncome:65 Over 10 to 14,999","HHIncome%:65 Over 10 to 14,999","HHIncome:65 Over 15 to 19,999","HHIncome%:65 Over 15 to 19,999","HHIncome:65 Over 20 to 24,999","HHIncome%:65 Over 20 to 24,999","HHIncome:65 Over 25 to 29,999","HHIncome%:65 Over 25 to 29,999","HHIncome:65 Over 30 to 34,999","HHIncome%:65 Over 30 to 34,999","HHIncome:65 Over 35 to 39,999","HHIncome%:65 Over 35 to 39,999","HHIncome:65 Over 40 to 44,999","HHIncome%:65 Over 40 to 44,999","HHIncome:65 Over 45 to 49,999","HHIncome%:65 Over 45 to 49,999","HHIncome:65 Over 50 to 59,999","HHIncome%:65 Over 50 to 59,999","HHIncome:65 Over 60 to 74,999","HHIncome%:65 Over 60 to 74,999","HHIncome:65 Over 75 to 99,999","HHIncome%:65 Over 75 to 99,999","HHIncome:65 Over 100 to 124,999","HHIncome%:65 Over 100 to 124,999","HHIncome:65 Over 125 to 149,999","HHIncome%:65 Over 125 to 149,999","HHIncome:65 Over 150 to 199,999","HHIncome%:65 Over 150 to 199,999",HHIncome:65 Over 200K or More,HHIncome%:65 Over 200K or More,CBAge:Cost Burdened Households Householder 65 Over,CBAge%:Households Cost Burdened Householder 65 Over,Homeownership:Householders Age 60 and Over,Homeownership:Homeowners Age 60 and Over,Homeownership%:Homeowners Age 60 and Over,Homeownership%:Age 60 and Over Householders Homeowners,Homeownership:Renters Age 60 and Over,Homeownership%:Renters Age 60 and Over,Homeownership%:Age 60 and Over Householders Renters,Homeownership:Householders Age 65 and Over,Homeownership:Homeowners Age 65 and Over,Homeownership%:Homeowners Age 65 and Over,Homeownership%:Age 65 and Over Householders Homeowners,Homeownership:Renters Age 65 and Over,Homeownership%:Renters Age 65 and Over,Homeownership%:Age 65 and Over Householders Renters,Poverty:Below Poverty Level 65 and Over,Poverty%:Below Poverty Level 65 and Over,Poverty%:65 and Over Below Poverty Level,Poverty:Population for Whom Poverty Status is Determined,Poverty: Population Below Poverty Level,Poverty%: Below Poverty Level,LivingSituation:Living Alone 65 Over,LivingSituation%:65 Over that are Living Alone,LivingSituation:Living with Spouse 65 Over,LivingSituation%:65 Over that are Living with Spouse,"Grandparents:Grandparents 60 Over, Primary Caregiver for Grandchild",Grandparents%:Grandparents 60 Over that are Primary Caregiver for Grandchild,Employment:In Labor Force 65 Over,Employment%:Total Population 65 Over In Labor Force,Employment%:Population in LF 65 Over that are Unemployed,SocialSecurity:Households with Social Security Income,SocialSecurity%:Households with Social Security Income
155,Tennessee,1172740.0,16.786806,50265.0,67097.0,37237.0,4.989642,47987.0,6.430109,49931.0,6.690599,46140.0,6.182616,43704.0,5.8562,38905.0,5.213149,37617.0,5.040561,34654.0,4.643528,35275.0,4.72674,59841.0,8.018508,72024.0,9.650992,84382.0,11.306925,53302.0,7.142302,33941.0,4.547988,32465.0,4.350209,38881.0,5.209933,189755.0,25.426579,1012117.0,820147.0,44.193884,81.032825,191970.0,21.027439,18.967175,746286.0,611075.0,32.927972,81.882147,135211.0,14.81034,18.117853,123622.0,13.14999,10.782715,6825291.0,940092.0,13.773654,324035.0,27.630592,297049.0,25.329485,17879.0,21.895781,210261.0,17.929038,3.01102,902742.0,32.60476


In [11]:
thelist = [data['HHIncome%:65 Over Less than 10,000'], data['HHIncome%:65 Over 10 to 14,999'], data['HHIncome%:65 Over 15 to 19,999'],
                data['HHIncome%:65 Over 20 to 24,999'], data['HHIncome%:65 Over 25 to 29,999'],
                data['HHIncome%:65 Over 30 to 34,999'], data['HHIncome%:65 Over 35 to 39,999'], data['HHIncome%:65 Over 40 to 44,999']]
data['HHIncome%:65 Over Making Less than 45,000'] = sum(thelist)

In [12]:
data

,NAME,Age:65 and Older,Age%:65 and Older,HHIncome:Median for Householder Under 65 Over,Median Household Income,"HHIncome:65 Over Less than 10,000","HHIncome%:65 Over Less than 10,000","HHIncome:65 Over 10 to 14,999","HHIncome%:65 Over 10 to 14,999","HHIncome:65 Over 15 to 19,999","HHIncome%:65 Over 15 to 19,999","HHIncome:65 Over 20 to 24,999","HHIncome%:65 Over 20 to 24,999","HHIncome:65 Over 25 to 29,999","HHIncome%:65 Over 25 to 29,999","HHIncome:65 Over 30 to 34,999","HHIncome%:65 Over 30 to 34,999","HHIncome:65 Over 35 to 39,999","HHIncome%:65 Over 35 to 39,999","HHIncome:65 Over 40 to 44,999","HHIncome%:65 Over 40 to 44,999","HHIncome:65 Over 45 to 49,999","HHIncome%:65 Over 45 to 49,999","HHIncome:65 Over 50 to 59,999","HHIncome%:65 Over 50 to 59,999","HHIncome:65 Over 60 to 74,999","HHIncome%:65 Over 60 to 74,999","HHIncome:65 Over 75 to 99,999","HHIncome%:65 Over 75 to 99,999","HHIncome:65 Over 100 to 124,999","HHIncome%:65 Over 100 to 124,999","HHIncome:65 Over 125 to 149,999","HHIncome%:65 Over 125 to 149,999","HHIncome:65 Over 150 to 199,999","HHIncome%:65 Over 150 to 199,999",HHIncome:65 Over 200K or More,HHIncome%:65 Over 200K or More,CBAge:Cost Burdened Households Householder 65 Over,CBAge%:Households Cost Burdened Householder 65 Over,Homeownership:Householders Age 60 and Over,Homeownership:Homeowners Age 60 and Over,Homeownership%:Homeowners Age 60 and Over,Homeownership%:Age 60 and Over Householders Homeowners,Homeownership:Renters Age 60 and Over,Homeownership%:Renters Age 60 and Over,Homeownership%:Age 60 and Over Householders Renters,Homeownership:Householders Age 65 and Over,Homeownership:Homeowners Age 65 and Over,Homeownership%:Homeowners Age 65 and Over,Homeownership%:Age 65 and Over Householders Homeowners,Homeownership:Renters Age 65 and Over,Homeownership%:Renters Age 65 and Over,Homeownership%:Age 65 and Over Householders Renters,Poverty:Below Poverty Level 65 and Over,Poverty%:Below Poverty Level 65 and Over,Poverty%:65 and Over Below Poverty Level,Poverty:Population for Whom Poverty Status is Determined,Poverty: Population Below Poverty Level,Poverty%: Below Poverty Level,LivingSituation:Living Alone 65 Over,LivingSituation%:65 Over that are Living Alone,LivingSituation:Living with Spouse 65 Over,LivingSituation%:65 Over that are Living with Spouse,"Grandparents:Grandparents 60 Over, Primary Caregiver for Grandchild",Grandparents%:Grandparents 60 Over that are Primary Caregiver for Grandchild,Employment:In Labor Force 65 Over,Employment%:Total Population 65 Over In Labor Force,Employment%:Population in LF 65 Over that are Unemployed,SocialSecurity:Households with Social Security Income,SocialSecurity%:Households with Social Security Income,"HHIncome%:65 Over Making Less than 45,000"
155,Tennessee,1172740.0,16.786806,50265.0,67097.0,37237.0,4.989642,47987.0,6.430109,49931.0,6.690599,46140.0,6.182616,43704.0,5.8562,38905.0,5.213149,37617.0,5.040561,34654.0,4.643528,35275.0,4.72674,59841.0,8.018508,72024.0,9.650992,84382.0,11.306925,53302.0,7.142302,33941.0,4.547988,32465.0,4.350209,38881.0,5.209933,189755.0,25.426579,1012117.0,820147.0,44.193884,81.032825,191970.0,21.027439,18.967175,746286.0,611075.0,32.927972,81.882147,135211.0,14.81034,18.117853,123622.0,13.14999,10.782715,6825291.0,940092.0,13.773654,324035.0,27.630592,297049.0,25.329485,17879.0,21.895781,210261.0,17.929038,3.01102,902742.0,32.60476,45.046403


In [14]:
data.to_csv('../Outputs/agewell_20235Yr.csv')